## Diffie Hellman in python

Below you can implement the DH protocol for key agreement


In [1]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend




In [2]:


# from the slides we know that 6 is a primitive root of 17
# TODO: IN THE END after it all works, select better numbers for q and alpha
publicP = 17;
publicG = 6;

print(publicP);
print(publicG);

#### private Alice inital part 

# TODO: select random aliceX (a)

print(f'[secret] Alice\'s X: {aliceX}');



## transmit aliceY -- you can assume that Bob will know it
print(f'Y can be transmited clear: {aliceY}');


#### private Bob  part 


# TODO: select random bobX (b)

print(f'[secret] Bob\'s X: {bobX}');

# TODO: Compute bobY = publicG ^ bobX mod publicP



## transmit bobY 
print(f'Y can be transmited clear: {bobY}');

#TODO: Bob Computes  bobK = (aliceY) ^ bobX mod publicP



#### private Alice final part 

#TODO: Alice Computes  aliceK = (bobY) ^ aliceX mod publicP



print(f'[secret] Alice\'s key: {aliceKey}');
print(f'[secret] Bob\'s   key: {aliceKey}');


17
6
[secret] Alice's X: 0
Y can be transmited clear: 0
[secret] Bob's X: 0
Y can be transmited clear: 0
[secret] Alice's key: 0
[secret] Bob's   key: 0


In [ ]:
#  Details about this implementation on : 
#    https://cryptography.io/en/latest/hazmat/primitives/asymmetric/dh/

In [8]:
%%time

# parameters should be agreed upon and can be shared publicly

parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())
# yellow

print(parameters.parameter_numbers())



CPU times: user 13.4 s, sys: 63.3 ms, total: 13.5 s
Wall time: 13.5 s


In [9]:
%%time

### --- Alice side ----
# Generate a private key for use in the exchange.

private_key_A = parameters.generate_private_key() # red
public_mixed_with_A  = private_key_A.public_key()  #orange


### --- B side ----

# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
private_key_B = parameters.generate_private_key() # Blue
public_mixed_with_B = private_key_B.public_key()  # light-blue-green :-) 



### --- A side ----

shared_key_B_then_A = private_key_A.exchange(public_mixed_with_B) # brown

print(f'[A-side] shared_key_B_then_A: {shared_key_B_then_A}') #brown


# Perform key derivation.
derived_key_B_then_A = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_B_then_A)


print(f'[A-side] derived_key_B_then_A: {derived_key_B_then_A}')







### --- B side ----

shared_key_A_then_B = private_key_B.exchange(public_mixed_with_A) #

print(f'[B-side] shared_key_A_then_B: {shared_key_A_then_B}') #brown




# Perform key derivation.
derived_key_A_then_B = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_A_then_B)


print(f'derived_key_A_then_B: {derived_key_A_then_B}')














[A-side] shared_key_B_then_A: b'GL\xcb\x13<\xf5*\xfbJ\x94_\xfd\xf5\xc2G\x9e\xf1\x91\x16\x93\xc9\xb2 |\x9c\x18f4>Y:\xef{\xed\xfeH\xf48\xf9\x8f%\xc5B2\xe3\xcejf\xe4\x9da\xc5U%L\x13\xb4\x89\xab\x18\xc6\xd1t\xfe\x94^N\x8f\xc1\xbf5\xa6\x96\x9d\x04=\x92\x1c\xca\xcf\x8c\x1a\xba\x1f\xc3\x8f\x95iY\x81:\x83\xcaV\x92?!\xf1)3\xd5\xda\xbeo6W\x0e\x0cn\xf1M\x1d\xfdj\t\xac&M\x11\x90;3\xc7\xb0\xd6SD@'
[A-side] derived_key_B_then_A: b'\xb3\xa2Tsw\x90\x1f\xb3vS\x9a\xa4\x15\xfd\xf2\x02[CK\xf3@\xd5"G\xef>o\xbe\xe6h\xca\x86'
[B-side] shared_key_A_then_B: b'GL\xcb\x13<\xf5*\xfbJ\x94_\xfd\xf5\xc2G\x9e\xf1\x91\x16\x93\xc9\xb2 |\x9c\x18f4>Y:\xef{\xed\xfeH\xf48\xf9\x8f%\xc5B2\xe3\xcejf\xe4\x9da\xc5U%L\x13\xb4\x89\xab\x18\xc6\xd1t\xfe\x94^N\x8f\xc1\xbf5\xa6\x96\x9d\x04=\x92\x1c\xca\xcf\x8c\x1a\xba\x1f\xc3\x8f\x95iY\x81:\x83\xcaV\x92?!\xf1)3\xd5\xda\xbeo6W\x0e\x0cn\xf1M\x1d\xfdj\t\xac&M\x11\x90;3\xc7\xb0\xd6SD@'
derived_key_A_then_B: b'\xb3\xa2Tsw\x90\x1f\xb3vS\x9a\xa4\x15\xfd\xf2\x02[CK\xf3@\xd5"G\xef>o\xbe\xe6h\